# Dridex (Doppeldridex) Loader

Sample (unpacked): `c7990f1e72fdfa84552f02f9d11cabb74251b0508291af5366fefcee646f9c91`

References:
- [UnpacMe Parent: 375167a16a6beeee52910d6424eb884c631ada9bcb9843809eecd0475718e549](https://www.unpac.me/results/35bcd307-47f0-427a-891c-96230dbcb766#/)
- [Malware Bazaar](https://bazaar.abuse.ch/sample/375167a16a6beeee52910d6424eb884c631ada9bcb9843809eecd0475718e549/)
- [Unpacked Sample (Malshare)](https://malshare.com/sample.php?action=detail&hash=c7990f1e72fdfa84552f02f9d11cabb74251b0508291af5366fefcee646f9c91)
- [Appgate dridex ioc extraction (prior research)](https://www.appgate.com/blog/reverse-engineering-dridex-and-automating-ioc-extraction)
- [API Resolving blog (Chuong Dong)](https://www.0ffset.net/reverse-engineering/malware-analysis/dridex-veh-api-obfuscation/)

## Helper Functions

In [2]:
def unhex(hex_string):
    import binascii
    if type(hex_string) == str:
        return binascii.unhexlify(hex_string.encode('utf-8'))
    else:
        return binascii.unhexlify(hex_string)

def tohex(data):
    import binascii
    if type(data) == str:
        return binascii.hexlify(data.encode('utf-8'))
    else:
        return binascii.hexlify(data)

## Known Data (Config) From Joe Sandbox

```
{
  "Version": 22201,
  "C2 list": [
    "103.42.56.15:443",
    "169.255.57.61:8116",
    "128.199.192.135:6602"
  ],
  "RC4 keys": [
    "s6ptqqQ96C42ODZyFUv32gPAVtURlPXrif68ogKV96MSXkXTs",
    "rZP9KOr8K1zzIudGa98GqFxsSgNXTDprMuKqtLAWjx1u1prqdOQuZjmv5"
  ]
}
```

## RC4 Decryption 

In [4]:
def rc4crypt(data, key):
    #If the input is a string convert to byte arrays
    if type(data) == str:
        data = data.encode('utf-8')
    if type(key) == str:
        key = key.encode('utf-8')
    x = 0
    box = list(range(256))
    for i in range(256):
        x = (x + box[i] + key[i % len(key)]) % 256
        box[i], box[x] = box[x], box[i]
    x = 0
    y = 0
    out = []
    for c in data:
        x = (x + 1) % 256
        y = (y + box[x]) % 256
        box[x], box[y] = box[y], box[x]
        out.append(c ^ box[(box[x] + box[y]) % 256])
    return bytes(out)


## Test String Decryption

In [17]:
data = unhex('63208a6488442f57a8414b4928cc1bb9660867a6c23f352adfa702ff121ceee14c69499849e264128bb21bda7fa2a8493244d4377003ec3caae05dc6a76ac7678acd82ca4b2ccceaf0602504576bc9c96223fda05aba4bbff6769f74bc7a3645839a41c4f610217d74d7d203cf38a89af0c05566f5e9301ea7d83f5102e8f324f90d3d38c0773e1a8db5dc6fb5808076269b686b40c6380c37de737d8838e530fac67a')
key = data[:48][::-1]
data = data[48:]

out = rc4crypt(data, key)
for s in out.split(b'\x00\x00'):
    print(s.replace(b'\x00',b'').decode('latin-1'))


ROOT\CIMV2
SELECT * FROM Win32_Fan
*.dll
*.exe
ntdll.dll




### APLib 

Credit: [Sandor Nemes (snemes)](https://github.com/snemes/aplib/blob/master/aplib.py)

In [22]:
import struct
from binascii import crc32
from io import BytesIO

__all__ = ['APLib', 'decompress']
__version__ = '0.6'
__author__ = 'Sandor Nemes'


class APLib(object):

    __slots__ = 'source', 'destination', 'tag', 'bitcount', 'strict'

    def __init__(self, source, strict=True):
        self.source = BytesIO(source)
        self.destination = bytearray()
        self.tag = 0
        self.bitcount = 0
        self.strict = bool(strict)

    def getbit(self):
        # check if tag is empty
        self.bitcount -= 1
        if self.bitcount < 0:
            # load next tag
            self.tag = ord(self.source.read(1))
            self.bitcount = 7

        # shift bit out of tag
        bit = self.tag >> 7 & 1
        self.tag <<= 1

        return bit

    def getgamma(self):
        result = 1

        # input gamma2-encoded bits
        while True:
            result = (result << 1) + self.getbit()
            if not self.getbit():
                break

        return result

    def depack(self):
        r0 = -1
        lwm = 0
        done = False

        try:

            # first byte verbatim
            self.destination += self.source.read(1)

            # main decompression loop
            while not done:
                if self.getbit():
                    if self.getbit():
                        if self.getbit():
                            offs = 0
                            for _ in range(4):
                                offs = (offs << 1) + self.getbit()

                            if offs:
                                self.destination.append(self.destination[-offs])
                            else:
                                self.destination.append(0)

                            lwm = 0
                        else:
                            offs = ord(self.source.read(1))
                            length = 2 + (offs & 1)
                            offs >>= 1

                            if offs:
                                for _ in range(length):
                                    self.destination.append(self.destination[-offs])
                            else:
                                done = True

                            r0 = offs
                            lwm = 1
                    else:
                        offs = self.getgamma()

                        if lwm == 0 and offs == 2:
                            offs = r0
                            length = self.getgamma()

                            for _ in range(length):
                                self.destination.append(self.destination[-offs])
                        else:
                            if lwm == 0:
                                offs -= 3
                            else:
                                offs -= 2

                            offs <<= 8
                            offs += ord(self.source.read(1))
                            length = self.getgamma()

                            if offs >= 32000:
                                length += 1
                            if offs >= 1280:
                                length += 1
                            if offs < 128:
                                length += 2

                            for _ in range(length):
                                self.destination.append(self.destination[-offs])

                            r0 = offs

                        lwm = 1
                else:
                    self.destination += self.source.read(1)
                    lwm = 0

        except (TypeError, IndexError):
            if self.strict:
                raise RuntimeError('aPLib decompression error')

        return bytes(self.destination)

    def pack(self):
        raise NotImplementedError


def aplib_decompress(data, strict=False):
    packed_size = None
    packed_crc = None
    orig_size = None
    orig_crc = None
    if data.startswith(b'AP32') and len(data) >= 24:
        # data has an aPLib header
        header_size, packed_size, packed_crc, orig_size, orig_crc = struct.unpack_from('=IIIII', data, 4)
        data = data[header_size : header_size + packed_size]
    if strict:
        if packed_size is not None and packed_size != len(data):
            raise RuntimeError('Packed data size is incorrect')
        if packed_crc is not None and packed_crc != crc32(data):
            raise RuntimeError('Packed data checksum is incorrect')
    result = APLib(data, strict=strict).depack()
    if strict:
        if orig_size is not None and orig_size != len(result):
            raise RuntimeError('Unpacked data size is incorrect')
        if orig_crc is not None and orig_crc != crc32(result):
            raise RuntimeError('Unpacked data checksum is incorrect')
    return result

## Test Data Decrypt

In [27]:
data = unhex('991fd7128ac6c8165a9d2daf172ef27944fd14f1fcaed3ff7f8cc4e02e434d439ee2becd4f076605a7bb5e20a2f4a73fa6c59f94cd7729eaa202576c1d33f0fc725fcdccd2b1d4dfb60c11f8d68aa7cc7acd78f97a9f9d777508d3d53c9c5fe6c64bb834f97680548dc6f093328097283261926ff58e662476ec8689b0b136ce9d7a8aa27a884c559fd5a60358f54c150587b89b01d6a3abdb2ec419029df02ce69c9997821cb14c8b2a4bbd74b747921650b98c93b50db1e557aa258b31397fe4ae9fde0f042dc856a2ed2865ccc639820ae7e94042f5deaa9c2d41f6a3034a251d954642e76b48002a2479ad96dea13bb77e74b1c28fbf5c61da648def0b3e99eda2a396594e3f59b3bc5d22c39f7aacc792a4e3db0bacdc5623dc2a333b8de3f27f3075ad2fd2a118b903e1f1f5fed45bde3fc1df0047f05ab27f0617a690440bba697ced12a1003eba57e49b69d40b9e4cbaf640ea8f03a57d2dc3507377ac4732a2eac76a10fc562139b3cc3cf616641feaeae1d580a4092f9ba3dbb0296ac8c7669d5c3635e2b2de5e05c2b2090c470b842be348f8aa35cac0b5a216d089689d88cb4f84d4a399c31d6acdb1bc7f86094fade26c3070bda7a37ae3254f94170f6163e8bef596c9025975d67bdcd1bf35cfdbed3ce789785f6d3c71b0472514dd2672c496d0fff1f5609ad632120c243930e481732ec5791ef33d0e7de20f925dc21f7aeeaea6a08af88ba3ae57863209f07d860cc0a54910043566ad4bdb4287a0fccce20685ec03f350a2fd22d7aaffb17304b5b7466fe317f624d91f53bf8d4d96369c810c0beae24c9307d555963ef0b3fd90fa1e7c99475a4b2c19047b356ca0e1220be20ba501f6e9c7e93a7803066d570be2d9f246a07ac151ec78ed665ab3ca827e6d03fe941b1ccc6e0d1a146c03c5751fe38e12781d34c90bf7636f9e5e322af63ba2e4fe71f6a2dffc05db028d5e3ac27a9a823c3c4471063f41a12263dd6378d32b602b6900d2de732ff215390ac8392fc6309f521e6a35d419dd8eb6a3be617fb48fc1ccbe1d640736b8fec3065a66b119fb7bdbf192bd9efd25c806a5cbb47d2fbcf3f5d2bd6a30a36617cc19bea06730e85417d006a44d35072fdb24dcb358c94b898a2fa80ab797113b636af50350a20cfb54ce441b15d26e62745354c223d342654e534a929ff5140b8bcba469e24794d795065ba8734b84d2063ae762d112fe5415266bab8ae35d10d68df561631acca2e69e61704bc4ee498c1c276c56251d3c7e5314e043bc6281d644e5e38d9fad56cb3284f2d2819b42423bd914bb7e75e46f47fce8f6da23c451d244bb807a5ce4399158f036b9364f6a91022ccc34dea34a2c00a0003c3fb773061c060a426c1ecd5aadbcaaccbe0146057cde291bf5f2968a58bb8d2a3793aa0431b6a5b20357eef157c0ed2340925c6ec1eb083d1edb3fe4fe72c2b9660507dee987cd5b612c5bf1048149c609feb0b658766af44f6368c6fd8ab2be56c4b1b102006681ab2423d9b47b6cda3445f7209c3b61d20f1a5855296a0d8bdae82b44a16e202b266bbd244c7918298b5d27ae28c775ff8f9b03220a9ace2f4e10474ae6db402f4deca0c54553cfabc348f8a3c45cb960b1272fb9434d0168af93a8d0201671b224fdca05281873e7cb5b3bb96a52657ed4de823c65422b44e1302509a8267ca8dc868ae3014add43e25c862210bc54118ab89893aa53f8da4adc008081a5dd95b50db9322f93a90b3daa5d36ce09680e6aad0f2a1bfe79d0136bf99bdeb13572ea6ed03e85f5e124eae9aad92310129a195f1ee97045db33f0acae6f13e7d3c909b9746901116c91b6d594574ba33192d244b82bdd03dc9c8bd88b6515257a95f85314037a6dd12291b9982d05eb87c83defcc47344f0200c4d395ad6fd43dce7475f284bb82e71efbef5aa1d58ce5dad95c5d584997e24629a6879afe7d06b6191564c9ace2ad8db1f64d0a0a6d5df4e7d5bd2868ef48347662e3d8ab3e579962c2a161588fc8fdde43bdf32ff4132aceed418999a5db02d3a38bacbc18360a42bc4f5a0ee6f4ae6d21f824ec237f2e9ca10ada9d9966a6d87a07f9313f6701702f12dd5b8cd1cce1e7cdeeac49bdfa3be62f1ec03d41ab5b7c7914d7ad5723d0e7acd8d2cac1d6473114e6562d08b27b4575864d22ca749ff148dd3478952cb03b663655fcd40cc26c4b444bada92fbbc6bbae16fc56316252dca3bf230c6b24d693f6f285da')

key = data[:48][::-1]
data = data[48:]

b64_data = rc4crypt(data, key)

import base64
import struct
cmp_data = base64.b64decode(b64_data)
decompressed_data_len = struct.unpack('<I',cmp_data[:4])
print("decompressed data len: %d" % decompressed_data_len)
out = aplib_decompress(cmp_data[4:])

SHELLCODE_FILE = '/tmp/dridex_shell1.bin'
open(SHELLCODE_FILE, 'wb').write(out)

decompressed data len: 1549


1549

## Shellcode Analysis

The loader checks to see if the process is 64 or 32 bit and then decryptes a shellcode blob and injects it along with a struct containing an IAT and other information.

[32bit shellcode (malshare)](https://malshare.com/sample.php?action=detail&hash=3f2bd4c55cd850faa873f90b4174452764f7adeba8f639df2fd3fbf591c80ca3)

**To be analyzed...**

## C2 Networking